In [45]:
import re
import requests
import json
from datetime import datetime
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from plotly.tools import mpl_to_plotly
from plotly.offline import iplot#,iplot_mpl,init_notebook_mode
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [2]:
from BaseObj import CallOption, PutOption, OptionType
from OptionFunc import getNextTTM, calculate_atm_strike

In [3]:
td = datetime.today()
rf = 1e-2

In [4]:
def getFutMisData(url, pay_load, ttm=""):
    
    header = {
        "Accept": "application/json, text/plain, */*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Content-Length": "144",
        "Content-Type": "application/json;charset=UTF-8",
        # Cookie: BIGipServerPOOL_MIS_TCP_80=!fkGW4xs9C3pI9ATNhh0qxO7/K4H+BIwsREOs/tPE3AFBZIQzxKDNDwv9VWmYsPlrEmfyb3vvOz/CxvQ=; _ga=GA1.3.1418395718.1656435390; _gid=GA1.3.1702948611.1663810804; _gat=1
        "Host": "mis.taifex.com.tw",
        "Origin": "https://mis.taifex.com.tw",
        "Referer": "https://mis.taifex.com.tw/futures/RegularSession/EquityIndices/Options/",
        "sec-ch-ua": '"Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }
    if 'ExpireMonth' in pay_load:
        pay_load['ExpireMonth'] = ttm
    res = requests.post(url, headers=header, data=json.dumps(pay_load))
    return res.json()

In [5]:
def readAllData():
    opt_payload = {
        "AscDesc": "A",
        "CID":"TXO",
        "ExpireMonth": "",
        "KindID": "1",
        "MarketType" : "0",
        'PageNo':"",
        "RowSize": "全部",
        "SortColumn":"",
        "SymbolType": "O"
    }

    ttm_url = 'https://mis.taifex.com.tw/futures/api/getCmdyMonthDDLItemByKind'
    ttm_data = getFutMisData(ttm_url, opt_payload, ttm="")
    ttm_strs = sorted([x['item'] for x in ttm_data['RtData']['Items'] if x['item'] != "現貨" and "W" not in x['item']])
    ttms = dict([(ttm_str, getNextTTM(datetime.strptime(ttm_str+"01", "%Y%m%d"))) for ttm_str in ttm_strs])
    
    opt_url = 'https://mis.taifex.com.tw/futures/api/getQuoteListOption'
    opt_data = {}
    for ttm_str in ttms.keys():
        opt_js = getFutMisData(opt_url, opt_payload, ttm=ttm_str)
        opt_data[ttm_str] = opt_js['RtData']['QuoteList']
    
    idx_url = 'https://mis.taifex.com.tw/futures/api/getQuoteDetail'
    idx_payload = {
        "SymbolID": ["TXF-S", "TXFJ2-F", "TXO-Q"]
    }
    idx_js = getFutMisData(idx_url, idx_payload)
    idx_data = idx_js['RtData']['QuoteList']
    
    fut_url = 'https://mis.taifex.com.tw/futures/api/getQuoteList'
    fut_payload = deepcopy(opt_payload)
    fut_payload.update({"SymbolType":"F", 'CID':"TXF"})
    fut_data = {}
    for ttm_str in ttms.keys():
        fut_js = getFutMisData(fut_url, fut_payload, ttm=ttm_str)
        fut_data[ttm_str] = fut_js['RtData']['QuoteList'][0]
    
    return idx_data[0], ttms, opt_data, fut_data

# Data Process

## ReadData

In [6]:
underlying, ttms, opt_data, fut_data = readAllData()
idx_price = float(underlying['CLastPrice'])

In [7]:
atm, spacing = calculate_atm_strike(idx_price, 'TXF')

## Create Option Object and Calculate IV

In [8]:
opt_maps = dict((ttm_str, {}) for ttm_str in ttms.keys())
exist_strikes = dict((ttm_str, []) for ttm_str in ttms.keys())
for ttm_str, tmp_opt_data in opt_data.items():
    for d in tmp_opt_data:
        premium = d["CLastPrice"]
        strike = d['StrikePrice']
        if not strike in exist_strikes[ttm_str]:
            exist_strikes[ttm_str].append(strike)
        if not len(premium):
            if len(d['CBidPrice1']) and len(d['CAskPrice1']):
                premium = (float(d['CBidPrice1']) + float(d['CAskPrice1'])) / 2
            elif len(d['CAskPrice1']):
                premium = float(d['CAskPrice1'])
            elif len(d['CBidPrice1']):
                premium = float(d['CBidPrice1'])
        opt_maps[ttm_str][strike] = opt_maps[ttm_str].get(strike, {})

        if d['CP'] == "C":
            if "Call" in opt_maps[ttm_str][strike].keys():
                opt_maps[ttm_str][strike]['Call'].UpdatePremium(underlying, premium)
            else:
                opt_maps[ttm_str][strike]['Call'] = CallOption(idx_price, int(strike), 
                                 TimeToMaturity=(ttms[ttm_str]-td).days, RiskFreeRate=rf, Premium=float(premium), Ticker=d['SymbolID'].split('-')[0])
        else:
            if "Put" in opt_maps[ttm_str][strike].keys():
                opt_maps[ttm_str][strike]['Put'].UpdatePremium(underlying, premium)
            else:
                opt_maps[ttm_str][strike]['Put'] = PutOption(idx_price, int(strike), 
                                 TimeToMaturity=(ttms[ttm_str]-td).days, RiskFreeRate=rf, Premium=float(premium), Ticker=d['SymbolID'].split('-')[0])

D:\SelfGit\SiteProject\StrategyDev\選擇權\BaseObj.py:104: RuntimeWarning:

divide by zero encountered in double_scalars

D:\SelfGit\SiteProject\StrategyDev\選擇權\BaseObj.py:63: RuntimeWarning:

invalid value encountered in double_scalars

D:\SelfGit\SiteProject\StrategyDev\選擇權\BaseObj.py:63: RuntimeWarning:

overflow encountered in double_scalars



## Plot IV Smile

In [46]:
strikes = {}
calls = {}
puts = {}
for TTM, v in opt_maps.items():
    for k, opt_ in v.items():
        strikes[TTM] = strikes.get(TTM, []) + [int(k)]
        calls[TTM] = calls.get(TTM, []) + [opt_['Call'].Sigma]
        puts[TTM] = puts.get(TTM, []) + [opt_['Put'].Sigma]
#         strikes.append(int(k))
#         calls.append(opt_['Call'].Sigma)
#         puts.append(opt_['Put'].Sigma)

fig,ax = plt.subplots(1, figsize=(13,6))
max_iv = 0
min_iv = 1
for TTM in strikes.keys():
    ax.plot(strikes[TTM], calls[TTM], label=f'Call IV at {TTM}', marker='o')
    ax.plot(strikes[TTM], puts[TTM], label=f'Put IV at {TTM}', marker='o')
    max_iv = max(max(calls[TTM] + puts[TTM]), max_iv)
    min_iv = min(min(calls[TTM] + puts[TTM]), min_iv)
# ax.vlines(underlying["CLastPrice"].split('.')[0], 0, max(calls+puts), linewidth=5)
ax.set_xlabel("履約價")
ax.set_ylabel("隱含波動率")
ax.set_title("波動率微笑曲線 (Volatility Smile)")
# ax.legend()
# plt.show()

plotly_fig = mpl_to_plotly(fig)
plotly_fig.add_trace(
    go.Scatter(
        name='價平',
        x = [atm, atm],
        y = [min_iv, max_iv],
        mode = "lines",
        marker = dict(color = 'rgba(80, 26, 80, 0.3)')
        )
)
plotly_fig.add_trace(
    go.Scatter(
        name='指數價格',
        x = [idx_price, idx_price],
        y = [min_iv, max_iv],
        mode = "lines+text",
        marker = dict(color = 'rgba(80, 80, 80, 0.3)'),
        text=idx_price
        )
)

plotly_fig['layout']['showlegend'] = True
iplot(plotly_fig)

# Plot Position Risk Analysis

## Caculate Margin

In [10]:
def getMargin(symbol, exchange, price:int or float, point:int or float, 
              margin:int or float=None, direction:int or float=None, underlyingPrice:int or float=None):
    """
    計算所需資金
        現貨 : 交割款
        期貨 : 原始保證金
        選擇權 : 原始保證金或權利金

    Params:
        symbol : 商品代碼
        exchange : 交易所
        price : 交易價格
        point : 價格還原現金之乘數
            例如:
                股票 : 1張 = 1,000股，10元的股票還原現金價值，即為10 *1,000 = 10,000元
                期貨 : 台指期1點200元，假設現在10,000點，則一口台股的價值為 200 * 10,000 = 2,000,000
        margin : 保證金佔資產價值比例(依期交所公告為準), default = None
        direction : 交易方向，進場(買或賣), default = None, 選擇權的買方不需保證金，賣方需支付保證金
        UnderlyingPrice : 標的資產價格，用於計算選擇權的保證金, default = None
    """
    try:
        if exchange == 'TFE':
            
            if isTXO(symbol): # 台指選
                print('is TXO')
                if direction != None:
                    if direction > 0:
                        return price * point
                    if direction < 0:
                        A, B, C = 48000, 24000, 4800
                        if symbol[-2] in 'ABCDEFGHIJKL': # Call
                            outofMoney = max(float(symbol[3:-2]) - underlyingPrice, 0) * point
                        else: # Put
                            outofMoney = max(underlyingPrice - float(symbol[3:-2]), 0) * point
                        return price * point + max(A - outofMoney, B)
            elif isTEO(symbol): # 電指選
                if direction != None:
                    if direction > 0: 
                        return price * point
                    if direction < 0:
                        A, B = 29000, 15000
                        if symbol[-2] in 'ABCDEFGHIJKL': # Call
                            outofMoney = max(float(symbol[3:-2]) - underlyingPrice, 0) * point
                        else: # Put
                            outofMoney = max(underlyingPrice - float(symbol[3:-2]), 0) * point
                        return price * point + max(A - outofMoney, B)
            elif isTFO(symbol): # 金指選
                if direction != None:
                    if direction > 0: 
                        return price * point
                    if direction < 0:
                        A, B = 17000, 9000
                        if symbol[-2] in 'ABCDEFGHIJKL': # Call
                            outofMoney = max(float(symbol[3:-2]) - underlyingPrice, 0) * point
                        else: # Put
                            outofMoney = max(underlyingPrice - float(symbol[3:-2]), 0) * point
                        return price * point + max(A - outofMoney, B)
#             elif len(symbol) == 5 and symbol[:3] in etfFuturesList: # ETF期貨
#                 return margin
            elif re.match('TX[0-9]*', symbol): # 大台指期
                return 182000
            elif re.match('TE[0-9]*', symbol): # 電指期
                return 180000
            elif re.match('TF[0-9]*', symbol): # 金指期
                return 79000
            elif re.match('MTX[0-9]*', symbol): # 小台指期
                return 46000
            elif isStockFuture(symbol): # 個股期貨
                if margin:
                    return price * point * margin
                else:
                    return price * point * 0.135
        else:
            return price * point
    except:
        pass
#         raise Exception(TransforException.GetException())

# @staticmethod
# def getCombinationMargin(symbols, positions, underlyingPrices):
#     """
#     計算選擇權組合策略之保證金
#     """
#     # TODO 計算選擇權組合策略之保證金
#     raise NotImplementedError('Should Implement Margin Calculation of Option Strategies')

# @staticmethod
def getMinimumTickFut(symbol, cost):
    if re.match(r'MTX[0-9]*', symbol): # 小台指期
        return 1
    elif re.match(r"TX[O0-9][0-9]*[A-Z][0-9]", symbol): # 台指選
        return Calculator.getMinimumTickOpt(cost)
    elif re.match(r"TEO[0-9]*[A-Z][0-9]", symbol): # 電指選
        return Calculator.getMinimumTickOptTE(cost)
    elif re.match(r"TFO[0-9]*[A-Z][0-9]", symbol): # 金指選
        return Calculator.getMinimumTickOptTF(cost)
    elif re.match(r'TX[0-9]*', symbol): # 大台指期
        return 1
    elif re.match(r'TE[0-9]*', symbol): # 電指期
        return 0.05
    elif re.match(r'TF[0-9]*', symbol): # 金指期
        return 0.2
    elif re.match(r"[A-Z][A-Z]F[0-9][0-9]", symbol): # 個股期貨
        return Calculator.getMinimumTick(cost)

# @staticmethod
def getMinimumTick(cost):
    if cost < 10:
        return 0.01
    elif cost < 50:
        return 0.05
    elif cost < 100:
        return 0.1
    elif cost < 500:
        return 0.5
    elif cost < 1000:
        return 1
    else:
        return 5

# @staticmethod
def getMinimumTickOpt(cost):
    if cost < 10:
        return 0.1
    elif cost < 50:
        return 0.5
    elif cost < 500:
        return 1
    elif cost < 1000:
        return 5
    else:
        return 10

# @staticmethod
def getMinimumTickOptTE(cost):
    if cost < 0.5:
        return 0.005
    elif cost < 2.5:
        return 0.025
    elif cost < 25:
        return 2.25
    elif cost < 50:
        return 0.25
    else:
        return 0.5

# @staticmethod
def getMinimumTickOptTF(cost):
    if cost < 2:
        return 0.02
    elif cost < 10:
        return 0.1
    elif cost < 100:
        return 0.2
    elif cost < 200:
        return 1
    else:
        return 2


# @staticmethod
def isTXO(symbol):
    return re.match(r"TX[O0-9][0-9]*[A-Z][0-9]", symbol)

# @staticmethod
def isTEO(symbol):
    return re.match(r"TEO[0-9]*[A-Z][0-9]", symbol)

# @staticmethod
def isTFO(symbol):
    return re.match(r"TFO[0-9]*[A-Z][0-9]", symbol)

# @staticmethod
def isStockFuture(symbol):
    return re.match(r"[A-Z][A-Z]F[0-9][0-9]", symbol)

## Multiple Position

In [11]:
def calculateFlatPoint(payoffs, sim_s):
    cross_zero = []
    for i, e in enumerate(payoffs):
        if not i :continue
        if np.sign(payoffs[i-1]) != np.sign(e):
            cross_zero.append((i-1, i))
        
    flat_points = []
    for idxs in cross_zero:
        flat_point = (0 - payoffs[idxs[0]]) * (sim_s[idxs[1]] - sim_s[idxs[0]])/(payoffs[idxs[1]] - payoffs[idxs[0]]) + sim_s[idxs[0]]
        flat_points.append(flat_point)
    return flat_points

In [55]:
def drawPositionPayoff(td, entry_idx, idx_price, ttms, spacing, positions, fut_positions=None, strategyName='自組'):
#     sim_s = [idx_price + i * spacing for i in range(-20, 21)]
    sim_s = [entry_idx * (1 + np.sign(i) / 100) ** abs(i) for i in range(-20, 21)]
    sim_percentage = [f"{i} %" for i in range(-20, 21)]
    combine_price = [0,] * len(sim_s)
    combine_price_expire = [0,] * len(sim_s)
    sum_current_price = 0
    min_ttm = None
    totalMargin = 0
    for position in positions:
        k, ttm_str, opt, pos, qty, cost_price = position
        c_price = cost_price if cost_price else opt.TheoryPrice()
        tmp_margin = getMargin(symbol=opt.Ticker, exchange="TFE", price=c_price, point=50, direction=pos, underlyingPrice=idx_price)
        print(opt.optionType, opt.Ticker, opt.TheoryPrice(), pos, qty, tmp_margin)
        totalMargin += round(tmp_margin)
        if min_ttm:
            min_ttm = min(min_ttm, ttm_str)
        else:
            min_ttm = ttm_str
        sum_current_price += opt.TheoryPrice() * qty * pos
        
        for i, s in enumerate(sim_s):
            current_price = cost_price if cost_price else opt.TheoryPrice()
            p = (opt.TheoryPrice(s, opt.K, opt.Sigma, opt.Tau, opt.RF) - current_price) * qty * pos
            tau_at_ttm = 1e-10 if (opt.Tau == (ttms[min_ttm] - td).days/opt.t_divisor) else (ttms[min_ttm] - td).days/opt.t_divisor
            p_expire = (opt.TheoryPrice(s, opt.K, opt.Sigma, tau_at_ttm, opt.RF) - current_price) * qty * pos
            combine_price[i] += p
            combine_price_expire[i] += p_expire
    
    for fut_pos in fut_positions:
        (ttm_str, fut_price, pos, qty, cost_price) = fut_pos
        print("Fut", fut_price, pos, qty)
        current_spread = fut_price - idx_price
        totalMargin += 46000 * qty
        for i, s in enumerate(sim_s):
            current_price = cost_price if cost_price else fut_price
            sim_fut_price = (s + current_spread - current_price) * qty * pos
            combine_price[i] += sim_fut_price
            combine_price_expire[i] += sim_fut_price
    
    fig,ax = plt.subplots(1, figsize=(13,6))
    ax.plot(sim_s, combine_price, label='到期前收益', marker='o')
    ax.plot(sim_s, combine_price_expire, label='到期日收益', marker='o')
    ax.set_xlabel("指數價格")
    ax.set_ylabel("選擇權價格")
    ax.set_title(f"不同指數價格下的選擇權組合Payoff ({strategyName}), 預估所需保證金: {totalMargin}")

    plotly_fig = mpl_to_plotly(fig)
    plotly_fig.add_trace(
        go.Scatter(
            name='價平',
            x = [atm, atm],
            y = [min(combine_price+combine_price_expire), max(combine_price+combine_price_expire)],
            mode = "lines",
            marker = dict(color = 'rgba(80, 26, 80, 0.3)')
            )
    )
    flat_points = calculateFlatPoint(combine_price_expire, sim_s)
    if flat_points:
        plotly_fig.add_trace(
            go.Scatter(
                name='損益兩平點',
                x = flat_points, 
                y = [0,] * len(flat_points), 
                mode = "markers+text",
                marker = dict(color = 'rgba(100, 26, 80, 0.7)'),
                marker_symbol='star',
                marker_size = 10,
                text = [str(round(x)) for x in flat_points],
                textposition="bottom center"
                )
        )
    plotly_fig.add_trace(
            go.Scatter(
                name='指數價格',
                x = [idx_price, idx_price], 
                y = [min(combine_price+combine_price_expire), max(combine_price+combine_price_expire)], 
                mode = "lines+text",
                marker = dict(color = 'rgba(100, 96, 80, 0.7)'),
#                 marker_symbol='star',
#                 marker_size = 10,
                text = [idx_price, idx_price],#[str(round(x)) for x in flat_points],
#                 textposition="bottom center"
                )
        )
    plotly_fig.add_trace(
        go.Scatter(
            name='0軸',
            x = [sim_s[0], sim_s[-1]],
            y = [0, 0],
            mode = "lines",
            marker = dict(color = 'rgba(80, 75, 20, 0.4)')
            )
    )
    plotly_fig.add_trace(
        go.Scatter(
            name='標的變動百分比',
            x = sim_s,
            y = combine_price,# [0,] * len(sim_s),
            mode = "markers+text",
#             marker = dict(color = 'rgba(80, 75, 20, 0.4)')
            text = sim_percentage,
            )
    )
    plotly_fig['layout']['showlegend'] = True
    iplot(plotly_fig)

In [58]:
# (strike, ttm_str, Call/Put, buy(1)/sell(-1), qty, cost_price)
strike = 17200
positions = [
            (strike, '202308', opt_maps['202308'][str(strike)]['Call'], -1, 1, opt_maps['202308'][str(strike)]['Call'].Premium),
            (strike, '202309', opt_maps['202309'][str(strike)]['Call'], 1, 3, opt_maps['202309'][str(strike)]['Call'].Premium),
            (strike, '202308', opt_maps['202308'][str(strike)]['Put'], -1, 1, opt_maps['202308'][str(strike)]['Put'].Premium),
            (strike, '202309', opt_maps['202309'][str(strike)]['Put'], 1, 3, opt_maps['202309'][str(strike)]['Put'].Premium),
#             (atm-200, '202307', opt_maps['202307'][str(atm-200)]['Put'], -1, 1, None),
#             (atm-200, '202308', opt_maps['202308'][str(atm-200)]['Put'], 1, 1, None),
#             (atm-500, opt_maps[str(atm-500)]['Put'], -1, 1),
]
# (ttm_str, current_price, buy(1)/sell(-1), qty, cost_price)
fut_positions = [
#     ('202308', float(fut_data['202308']['CLastPrice']), 1, 1, float(fut_data['202308']['CLastPrice']))
]
drawPositionPayoff(td, strike, idx_price, ttms, spacing, positions, fut_positions=fut_positions, strategyName='時間價差')

is TXO
OptionType.CALL TXO17200H3 230.032188006252 -1 1 57744.49999999997
is TXO
OptionType.CALL TXO17200I3 363.04075011176883 1 3 18150.0
is TXO
OptionType.PUT TXO17200T3 299.0400118316211 -1 1 62950.0
is TXO
OptionType.PUT TXO17200U3 475.02592101345 1 3 23750.0


In [35]:
# (strike, ttm_str, Call/Put, buy(1)/sell(-1), qty, cost_price)
strike = 16600
positions = [
            (strike, '202307', opt_maps['202307'][str(strike)]['Call'], 1, 1, 181),
            (strike, '202308', opt_maps['202308'][str(strike)]['Call'], -1, 5, 293),
#             (atm-200, '202210', opt_maps['202210'][str(atm-200)]['Put'], -1, 1, None),
#             (atm-200, '202211', opt_maps['202211'][str(atm-200)]['Put'], 1, 3, None),
#             (atm-500, opt_maps[str(atm-500)]['Put'], -1, 1),
]
# (ttm_str, current_price, buy(1)/sell(-1), qty, cost_price)
fut_positions = [
    ('202307', float(fut_data['202307']['CLastPrice']), 1, 1, 16623)
]
drawPositionPayoff(td, strike, idx_price, ttms, spacing, positions, fut_positions=fut_positions, strategyName='時間價差')

is TXO
OptionType.CALL TXO16600G3 181.04454548242393 1 1 9050
is TXO
OptionType.CALL TXO16600H3 293.0186620038912 -1 5 62650
Fut 16623.0 1 1


In [41]:
62900 + 427*50*3 + 46000

172950